In [38]:
from math import sqrt, log
import numpy as np
from copy import deepcopy
from random import choice

In [39]:
class MonteCarloTree:
    def __init__(self, game_state, player_no, num_iter=1000, parent=None, action=None, tie_score=0):
        self.game_state = game_state
        self.parent = parent
        self.player_no = player_no
        self.action = action
        self.num_iter = num_iter
        self.tie_score = tie_score
        self.children = []
        self.visits = 0
        self._results = 0
        self.untried_actions = self.get_legal_actions()
    
    def get_legal_actions(self):
        # Return a list of all legal actions in the current game_state
        actions = []
        
        if self.checkvictory(self.game_state) != 0:
            return actions
        
        for row in range(3):
            for col in range(3):
                if self.game_state[row][col]==0:
                    actions.append(row*3+col+1)
        return actions
    
    def play(self, action, game_state):
        row = (action - 1) // 3
        col = (action - 1) % 3
        if self.player_no==1:
            game_state[row][col] = 1
        else:
            game_state[row][col] = -1
        return game_state
    
    def score(self, total, c):
        n = self.visits
        if n==0 and c>0:
            return 10
        return self._results / n + c * sqrt((log(total)) / n)
    
    def add_child(self, action):
        # Add a new child MonteCarloTree for the given action
        if self.player_no == 1:
            new_player = 2
        else:
            new_player = 1
        new_game_state = self.play(action, deepcopy(self.game_state))
        child = MonteCarloTree(new_game_state, parent=self, player_no=new_player, action=action)
        self.children.append(child)
        return child
    
    def get_move(self, win_score):
        
        for i in range(3):
            if abs(sum(self.game_state[i])) == 2 and sum(self.game_state[i]) == win_score:  # Check each row
                for k in range(3):
                    if self.game_state[i][k] == 0:
                        return i*3 + k + 1

            if abs(sum(self.game_state[j][i] for j in range(3))) == 2 and sum(self.game_state[j][i] for j in range(3)) == win_score:  # Check each column
                for k in range(3):
                    if self.game_state[k][i] == 0:
                        return k*3+i+1
    
        # Check diagonals
        if abs(self.game_state[0][0] + self.game_state[1][1] + self.game_state[2][2]) == 2 and self.game_state[0][0] + self.game_state[1][1] + self.game_state[2][2] == win_score:
            for k in range(3):
                if self.game_state[k][k] == 0:
                    return k*3+k+1

        if abs(self.game_state[0][2] + self.game_state[1][1] + self.game_state[2][0]) == 2 and self.game_state[0][2] + self.game_state[1][1] + self.game_state[2][0] == win_score:
            for k in range(3):
                if self.game_state[k][2-k] == 0:
                    return k*3+3-k
        
        return -1
    
    def rollout_policy(self):
        
        if self.player_no == 1:
            win_score = 2
        else:
            win_score = -2
        
        win_move = self.get_move(win_score)
        
        if win_move in self.untried_actions:
            return win_move
        
        loss_move = self.get_move(-win_score)

        if loss_move in self.untried_actions:
            return loss_move
        
        return choice(self.untried_actions)
    
    def expand(self):
        action = self.rollout_policy()
        self.untried_actions.remove(action)
        return self.add_child(action)
    
    def rollout(self):
        current = self
        while current.checkvictory(current.game_state) == 0:
            current = current.expand()
        if current.checkvictory(current.game_state) == -2:
            return self.tie_score
        if current.checkvictory(current.game_state) == 1:
            return 1
        return -1
    
    def best_child(self, c):
        total = 0
        for child in self.children:
            total += child.visits
        scores = []
        pos = {}
        for child in self.children:
            scores.append(child.score(total, c=c))
            pos[child.action] = child.score(total, c=c)
        return self.children[np.argmax(scores)]
    
    def best_action(self):
        
        has_win_policy = self.get_move(2)
        if has_win_policy != -1:
            return has_win_policy
        has_loss_policy = self.get_move(-2)
        if has_loss_policy != -1:
            return has_loss_policy
        
        for _ in range(self.num_iter):
            current = self.select_child()
            result = current.rollout()
            current.backpropagate(result)
            
        return self.best_child(0).action
    
    def select_child(self):
        current = self
        while current.checkvictory(current.game_state) == 0:
            if current.untried_actions != []:
                return current.expand()
            else:
                current = current.best_child(sqrt(2))
        return current
    
    def checkvictory(self, game_state):
        
        for i in range(3):
            if abs(sum(game_state[i])) == 3:  # Check each row
                return game_state[i][0]
            if abs(sum(game_state[j][i] for j in range(3))) == 3:  # Check each column
                return game_state[0][i]

        # Check diagonals
        if abs(game_state[0][0] + game_state[1][1] + game_state[2][2]) == 3:
            return game_state[0][0]
        
        if abs(game_state[0][2] + game_state[1][1] + game_state[2][0]) == 3:
            return game_state[0][2]
        
        for i in range(3):
            for j in range(3):
                if game_state[i][j] == 0:
                    return 0
        return -2
        
    def backpropagate(self, result):
        # Update the MonteCarloTree's scores
        self.visits += 1
        self._results += result
        if self.parent:
            self.parent.backpropagate(result)


In [40]:
class TicTacToe:
    def __init__(self, tie_score=0):
        self.game_state = np.zeros(dtype=int, shape=(3,3))
        self.tie_score = tie_score
        
    def get_legal_actions(self):
        # Return a list of all legal actions in the current game_state
        actions = []
        
        if self._checkGameOver() != 0:
            return actions
        
        for row in range(3):
            for col in range(3):
                if self.game_state[row][col]==0:
                    actions.append(row*3+col+1)
        return actions
    
    def get_move(self, score):
        
        for i in range(3):
            if abs(sum(self.game_state[i])) == 2 and sum(self.game_state[i]) == score:  # Check each row
                for k in range(3):
                    if self.game_state[i][k] == 0:
                        return i*3 + k + 1

            if abs(sum(self.game_state[j][i] for j in range(3))) == 2 and sum(self.game_state[j][i] for j in range(3)) == score:  # Check each column
                for k in range(3):
                    if self.game_state[k][i] == 0:
                        return k*3+i+1
    
        # Check diagonals
        if abs(self.game_state[0][0] + self.game_state[1][1] + self.game_state[2][2]) == 2 and self.game_state[0][0] + self.game_state[1][1] + self.game_state[2][2] == score:
            for k in range(3):
                if self.game_state[k][k] == 0:
                    return k*3+k+1

        if abs(self.game_state[0][2] + self.game_state[1][1] + self.game_state[2][0]) == 2 and self.game_state[0][2] + self.game_state[1][1] + self.game_state[2][0] == score:
            for k in range(3):
                if self.game_state[k][2-k] == 0:
                    return k*3+3-k
        
        return -1
    
    def policy(self):
        win_score = -2
        loss_score = 2
        untried_actions = self.get_legal_actions()
        
        win_move = self.get_move(win_score)
        
        if win_move in untried_actions:
            return win_move
        
        loss_move = self.get_move(loss_score)

        if loss_move in untried_actions:
            return loss_move
        
        return choice(untried_actions)
    
    def _checkGameOver(self):
        for i in range(3):
            if abs(sum(self.game_state[i])) == 3:  # Check each row
                return self.game_state[i][0]
            if abs(sum(self.game_state[j][i] for j in range(3))) == 3:  # Check each column
                return self.game_state[0][i]
        
        # Check diagonals
        if abs(self.game_state[0][0] + self.game_state[1][1] + self.game_state[2][2]) == 3:
            return self.game_state[0][0]
        
        if abs(self.game_state[0][2] + self.game_state[1][1] + self.game_state[2][0]) == 3:
            return self.game_state[0][2]
        
        for i in range(3):
            for j in range(3):
                if self.game_state[i][j] == 0:
                    return 0
        return -2
    
    def startGame(self):
        self.game_state = np.zeros(dtype=int, shape=(3,3))
        
    def printGame(self):
        for i in self.game_state:
            line = []
            for j in i:
                if (j==1):
                    line.append("X")
                elif (j==-1):
                    line.append("O")
                else:
                    line.append(" ")
                if (len(line) < 6):
                    line.append("|")
            print(''.join(line))
            print("-------")
            
    def evaluateAIAgainstRandom(self):
        wins = 0
        ties = 0
        loss = 0
        for _ in range(1000):        
            self.startGame()
            turn = choice([0,1])
            while(self._checkGameOver() == 0):
                player_no = 1 if turn%2==0 else 2
                if (player_no==1):
                    pos = MonteCarloTree(self.game_state, player_no=1, tie_score=self.tie_score).best_action()
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = 1
                else:
                    pos = choice(self.get_legal_actions())
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = -1
                turn +=1
            state = self._checkGameOver()
            if (state==1):
                wins += 1
            elif (state==-1):
                loss += 1
            else:
                ties += 1
        return {'wins':wins, 'losses': loss, 'ties': ties}
    
    def evaluatePolicyAgainstRandom(self):
        wins = 0
        ties = 0
        loss = 0
        for _ in range(1000):        
            self.startGame()
            turn = choice([0,1])
            while(self._checkGameOver() == 0):
                player_no = 1 if turn%2==0 else 2
                if (player_no==1):
                    pos = choice(self.get_legal_actions())
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = 1
                else:
                    pos = self.policy()
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = -1
                turn +=1
            state = self._checkGameOver()
            if (state==1):
                loss += 1
            elif (state==-1):
                wins += 1
            else:
                ties += 1
        return {'wins':wins, 'losses': loss, 'ties': ties}
    
    def evaluateAIAgainstPolicy(self):
        wins = 0
        ties = 0
        loss = 0
        for _ in range(1000):        
            self.startGame()
            turn = choice([0,1])
            while(self._checkGameOver() == 0):
                player_no = 1 if turn%2==0 else 2
                if (player_no==1):
                    pos = MonteCarloTree(self.game_state, player_no=1, tie_score=self.tie_score).best_action()
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = 1
                else:
                    pos = self.policy()
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = -1
                turn +=1
            state = self._checkGameOver()
            if (state==1):
                wins += 1
            elif (state==-1):
                loss += 1
            else:
                ties += 1
        return {'wins':wins, 'losses': loss, 'ties': ties}
            
    def playGame(self, first):        
        self.startGame()
        self.printGame()
        turn = 1 if first else 0
        while(self._checkGameOver() == 0):
            print(f"Turn Number:  {turn+1}")
            waitInput = True
            while (waitInput):
                player_no = 1 if turn%2==0 else 2
                if (player_no==1):
                    pos = MonteCarloTree(self.game_state, player_no=1, tie_score=self.tie_score).best_action()
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    self.game_state[row][col] = 1
                else:
                    pos = int(input(f"Player, please input your move: "))
                    row = (pos - 1) // 3
                    col = (pos - 1) % 3
                    if (pos>9 or pos<1):
                        print("Invalid move! Please enter a number between 1 to 9")
                        continue
                    elif (self.game_state[row][col]!=0):
                        print("Invalid move! The tile is already taken")
                        continue
                    self.game_state[row][col] = -1
                waitInput = False
            self.printGame()
            turn +=1
            print("---------------------")
        state = self._checkGameOver()
        if (state==1):
            print("AI wins!")
        elif (state==-1):
            print("Player wins!")
        else:
            print("It's a tie!")
            

In [41]:
TicTacToe().evaluateAIAgainstRandom()

{'wins': 915, 'losses': 16, 'ties': 69}

In [42]:
TicTacToe().evaluatePolicyAgainstRandom()

{'wins': 806, 'losses': 30, 'ties': 164}

In [43]:
TicTacToe().evaluateAIAgainstPolicy()

{'wins': 291, 'losses': 244, 'ties': 465}

In [44]:
TicTacToe(tie_score=0.333).evaluateAIAgainstRandom()

{'wins': 930, 'losses': 19, 'ties': 51}

In [73]:
TicTacToe(tie_score=0.333).evaluateAIAgainstPolicy()

{'wins': 291, 'losses': 207, 'ties': 502}

In [49]:
TicTacToe(tie_score=0.333).playGame(first=False)

 | | |
-------
 | | |
-------
 | | |
-------
Turn Number:  1
 | | |
-------
 |X| |
-------
 | | |
-------
---------------------
Turn Number:  2
 |O| |
-------
 |X| |
-------
 | | |
-------
---------------------
Turn Number:  3
X|O| |
-------
 |X| |
-------
 | | |
-------
---------------------
Turn Number:  4
X|O| |
-------
 |X| |
-------
 | |O|
-------
---------------------
Turn Number:  5
X|O| |
-------
 |X| |
-------
X| |O|
-------
---------------------
Turn Number:  6
X|O| |
-------
O|X| |
-------
X| |O|
-------
---------------------
Turn Number:  7
X|O|X|
-------
O|X| |
-------
X| |O|
-------
---------------------
AI wins!


In [50]:
TicTacToe(tie_score=0.333).playGame(first=True)

 | | |
-------
 | | |
-------
 | | |
-------
Turn Number:  2
 | | |
-------
 |O| |
-------
 | | |
-------
---------------------
Turn Number:  3
 | | |
-------
 |O| |
-------
 | |X|
-------
---------------------
Turn Number:  4
O| | |
-------
 |O| |
-------
 | |X|
-------
---------------------
Turn Number:  5
O| | |
-------
 |O| |
-------
X| |X|
-------
---------------------
Turn Number:  6
O| | |
-------
 |O| |
-------
X|O|X|
-------
---------------------
Turn Number:  7
O|X| |
-------
 |O| |
-------
X|O|X|
-------
---------------------
Turn Number:  8
O|X|O|
-------
 |O| |
-------
X|O|X|
-------
---------------------
Turn Number:  9
O|X|O|
-------
 |O|X|
-------
X|O|X|
-------
---------------------
Turn Number:  10
O|X|O|
-------
O|O|X|
-------
X|O|X|
-------
---------------------
It's a tie!


In [52]:
TicTacToe(tie_score=0.333).playGame(first=True)

 | | |
-------
 | | |
-------
 | | |
-------
Turn Number:  2
 | | |
-------
 |O| |
-------
 | | |
-------
---------------------
Turn Number:  3
 | | |
-------
 |O| |
-------
X| | |
-------
---------------------
Turn Number:  4
 | |O|
-------
 |O| |
-------
X| | |
-------
---------------------
Turn Number:  5
X| |O|
-------
 |O| |
-------
X| | |
-------
---------------------
Turn Number:  6
X| |O|
-------
O|O| |
-------
X| | |
-------
---------------------
Turn Number:  7
X| |O|
-------
O|O|X|
-------
X| | |
-------
---------------------
Turn Number:  8
X|O|O|
-------
O|O|X|
-------
X| | |
-------
---------------------
Turn Number:  9
X|O|O|
-------
O|O|X|
-------
X|X| |
-------
---------------------
Turn Number:  10
X|O|O|
-------
O|O|X|
-------
X|X|O|
-------
---------------------
It's a tie!
